In [1]:
!pip3 install mediapipe

In [3]:
! pip install protobuf==3.19.4

  Using cached protobuf-3.19.4-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.0
    Uninstalling protobuf-3.19.0:
      Successfully uninstalled protobuf-3.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnxruntime 1.13.1 requires sympy, which is not installed.
streamlit 1.12.0 requires blinker>=1.0.0, which is not installed.
streamlit 1.12.0 requires click>=7.0, which is not installed.
streamlit 1.12.0 requires pydeck>=0.1.dev5, which is not installed.
streamlit 1.12.0 requires pympler>=0.9, which is not installed.
streamlit 1.12.0 requires rich>=10.11.0, which is not installed.
streamlit 1.12.0 requires semver, which is not installed.
streamlit 1.12.0 requires toml, which is not installed.
streamlit 1.12.0 requires tzlocal>=1.1, which is not installed.
streamlit 1.12.0 requires validators>=0

Then download an off-the-shelf model bundle. Check out the MediaPipe documentation for more information about this model bundle.

In [2]:
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

Visualization utilities

In [5]:
import mediapipe

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (/Users/nori/.pyenv/versions/3.9.16/lib/python3.9/site-packages/google/protobuf/internal/__init__.py)

In [6]:
import mediapipe

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (/Users/nori/.pyenv/versions/3.9.16/lib/python3.9/site-packages/google/protobuf/internal/__init__.py)

In [7]:
#@markdown To better demonstrate the Pose Landmarker API, we have created a set of visualization tools that will be used in this colab. These will draw the landmarks on a detect person, as well as the expected connections between those markers.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (/Users/nori/.pyenv/versions/3.9.16/lib/python3.9/site-packages/google/protobuf/internal/__init__.py)